In [1]:
import json
import ast
import string
import re

In [2]:
import pandas as pd

In [3]:
failed_sample = pd.read_json("./failed_extraction.json", lines=True)

In [4]:
datapoint = failed_sample.iloc[9]

In [122]:
original = datapoint['sentence']
original

"Responsabilités principales Établir un contact téléphonique personnalisé avec le client créer un climat de confiance favorisant la vente d'un contrat d'assurance entreprise tout en respectant les politiques de souscription de l'entreprise."

In [258]:
original = "the ability to operate a a senior manager in a dynamic commercially focus environment advance use of m excel and powerpoint"

In [259]:
generated = datapoint['extracted_skills'][0]
generated

"Responsabilités principales @@Établir un contact téléphonique personnalisé avec le client, créer un climat de confiance favorisant la vente d'un contrat d'assurance entreprise tout en respectant les politiques de souscription de l'entreprise.##"

In [260]:
generated = "the ability to operate as a @@senior manager## in a @@dynamic## commercially focus environment advance use of @@m excel## and @@powerpoint##"

In [261]:
def extract_spans(sentence):
    pattern = r'@@(.*?)##'
    spans = re.findall(pattern, sentence)
    return spans

def postprocess_ner_prompt(original, generation):
    print("======= INSIDE POSTPROCESS =======")
    print(f"ORIGINAL: {original}")
    print(f"GENERATION: {generation}")

    puntuation_list = ['.', ',', '!', '?', ';', ':', '\'', '"', '/', '(', ')', '[', ']', '{', '}']

    if generation.endswith("##") and generation[-3] in puntuation_list:
        if generation[-4] == ' ':
            generation = generation[:-4] + "##" + generation[-3]
        else:
            generation = generation[:-3] + "##" + generation[-3]
    if original[-1] not in puntuation_list and generation[-1] in puntuation_list:
        generation = generation[:-1]

    extracted_spans = extract_spans(generation)

    pattern = r"(\w|##)([.,!?;:')\]}\"\/](?:##)?)|((?:@@)?[.,!?;:'(\[{\"\/])(\w)"

    # add spaces around punctuation
    cleaned_generation = re.sub(pattern, r'\1 \2 \3 \4', generation)
    # remove duplicated spaces
    cleaned_generation = re.sub(r'\s+', ' ', cleaned_generation).rstrip()

    if original[-1] in puntuation_list and original[-2] != ' ':
        generation = generation[:-1] 

    print(f"CLEANED: {cleaned_generation}")

    mismatched = False 

    original_fixed = []
    generation_fixed = []
    original_idx = 0
    generated_idx = 0

    while original_idx < len(original) and generated_idx < len(cleaned_generation):
        original_char = original[original_idx]
        generated_char = cleaned_generation[generated_idx]

        # Check if the characters match
        if original_char == generated_char:
            original_fixed.append(original_char)
            generation_fixed.append(generated_char)
            original_idx += 1
            generated_idx += 1

        else:
            if generated_char == "#" or generated_char == "@":
                generation_fixed.append(generated_char)
                generated_idx += 1
            
            elif generated_char == ' ':
                if original_char in puntuation_list and cleaned_generation[generated_idx + 1] == original_char \
                    or cleaned_generation[generated_idx - 1] in puntuation_list and cleaned_generation[generated_idx + 1] == original_char:
                    generation_fixed.append(cleaned_generation[generated_idx + 1])
                    original_fixed.append(original_char)
                    generated_idx += 2
                    original_idx += 1
               
                elif cleaned_generation[generated_idx - 1] in puntuation_list and \
                    cleaned_generation[generated_idx + 1] == '@' and cleaned_generation[generated_idx + 3] == original_char: 
                    generation_fixed.extend(['@', '@'])
                    generation_fixed.append(cleaned_generation[generated_idx + 3])
                    original_fixed.append(original_char)
                    generated_idx += 4
                    original_idx += 1 
 
                else:
                    mismatched = True
                    break
            
            elif generated_char in puntuation_list:
                if original_char == ' ' and original[original_idx + 1] == generated_char:
                    generation_fixed.append(' ')
                    generation_fixed.append(generated_char)
                    original_fixed.append(original_char)
                    original_fixed.append(original[original_idx + 1])
                    generated_idx += 1
                    original_idx += 2
                elif original_char in string.ascii_lowercase:
                    if cleaned_generation[generated_idx + 2] == original_char: # random punctuation assertion
                        generation_fixed.append(cleaned_generation[generated_idx + 2])
                        original_fixed.append(original_char)
                        generated_idx += 3
                        original_idx += 1
                    elif cleaned_generation[generated_idx + 2] == "@" and cleaned_generation[generated_idx + 4] == original_char:
                        generation_fixed.extend(['@', '@'])
                        generation_fixed.append(cleaned_generation[generated_idx + 4])
                        original_fixed.append(original_char) 
                        generated_idx += 5
                        original_idx += 1 

            elif generated_char in string.ascii_lowercase and original_char == ' ': # check for random spaces in original
                if (cleaned_generation[generated_idx-4:generated_idx] == original[original_idx-4:original_idx]) \
                and (cleaned_generation[generated_idx:generated_idx+4] == original[original_idx+1:original_idx+5]):
                   generation_fixed.append(original_char)
                   original_idx += 1 
                else:
                    mismatched = True
                    break
                     

            elif original_char not in string.ascii_lowercase and generated_char not in string.ascii_letters:
                generated_idx += 1
                original_idx += 1
            else:
                mismatched = True
                break
        
        # print(f"{''.join(original_fixed)}\n{''.join(generation_fixed)}\n\n")
        # print
    print(original[original_idx:])
    original_fixed.extend(original[original_idx:])
    generation_fixed.extend(cleaned_generation[generated_idx:])
   
    generated_fixed_str = ''.join(generation_fixed)

    if len(original.split()) != len(generated_fixed_str.split()):
        mismatched = True

    extracted_spans = [ent for ent in extracted_spans if ent in original]

    print(f"UPDATED: {generated_fixed_str}")
    print(f"mismatched: {mismatched}")
    print("================================")
    return generated_fixed_str, mismatched, extracted_spans

In [262]:
generation, mismatched, spans = postprocess_ner_prompt(original, generated)

======= INSIDE POSTPROCESS =======
ORIGINAL: the ability to operate a a senior manager in a dynamic commercially focus environment advance use of m excel and powerpoint
GENERATION: the ability to operate as a @@senior manager## in a @@dynamic## commercially focus environment advance use of @@m excel## and @@powerpoint##
CLEANED: the ability to operate as a @@senior manager## in a @@dynamic## commercially focus environment advance use of @@m excel## and @@powerpoint##
 a senior manager in a dynamic commercially focus environment advance use of m excel and powerpoint
UPDATED: the ability to operate as a @@senior manager## in a @@dynamic## commercially focus environment advance use of @@m excel## and @@powerpoint##
mismatched: True


In [225]:
spans

['mettant à profit votre capacité à convaincre']

In [226]:
len(generation.split())

33

In [227]:
generation

'@@Vous assurez la prise en charge des membres et clients, @@analysez leurs besoins## et les @@accompagnez dans le choix des protections ap propriées## et ce, en @@mettant à profit votre capacité à convaincre##.'

In [228]:
mismatched

False

In [229]:
failed_sentences = set()
with open("failed_extraction.json", "r") as readfile:
    for line in readfile:
        instance = json.loads(line)
        failed_sentences.add(instance['sentence']) 

In [230]:
failed_sentences

{'IT skills allied to a high standard of work output , you exhibit a high degree of professionalism , team focus and a selfdisciplined approach to your own workload and organisation .',
 "SOMMAIRE DES FONCTIONS Sous la supervision du Directeur, sélection des risques & du Directeur de l'Administation, le tituliare assiste le service dans l'élaboration des différentes tâches administratives reliées tel que: - Reçoit et distribue la correspondance quotidiennement - Vérifier la facturation de nos fournisseurs et faire approuver le versement - Répondre aux appels des agents généraux (bilinguisme essentiel); - Envoi de rapports divers (direction, délai de traitement et service etc…); - Commande de matériels promotionnels et fournitures de toutes sortes - Transcription, correction et vérification de lettres - Effectue tout travail de bureau requis par la ligne d’affaires Le titulaire devra régulièrement remplacer la réceptionniste durant ses pauses, de même que le vendredi ou autres absences.